In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import getpass

from kgforge.core import KnowledgeGraphForge

from inference_tools.utils import fetch_rules, check_premises, apply_rule, get_rule_parameters

In [23]:
ENDPOINT = "https://bbp.epfl.ch/nexus/v1"

In [24]:
TOKEN = getpass.getpass()

········


In [25]:
# This could be replaced by any util kg-inference-api implements to handle user forge sessions
FORGE_SESSIONS = {}
def allocate_forge_session(org, project):
    if (org, project) not in FORGE_SESSIONS:
        session = KnowledgeGraphForge(
            "../../configs/new-forge-config.yaml",
            endpoint=ENDPOINT,
            token=TOKEN, 
            bucket=f"{org}/{project}")
        FORGE_SESSIONS[(org, project)] = session
    return FORGE_SESSIONS[(org, project)]

In [26]:
sscx_forge = allocate_forge_session("public", "sscx")

In [ ]:
resources = sscx_forge.sparql("""
SELECT ?morph ?trace
WHERE {
    ?morph a <https://neuroshapes.org/NeuronMorphology> ;
        <https://neuroshapes.org/derivation>/<http://www.w3.org/ns/prov#entity> ?cell.
    ?cell a <https://neuroshapes.org/PatchedCell> .
    ?trace a <https://neuroshapes.org/Trace> ;
           <https://neuroshapes.org/derivation>/<http://www.w3.org/ns/prov#entity> ?cell.
}
""", limit=None)

In [24]:
simulation_traces = sscx_forge.sparql("""
SELECT ?trace ?mtype
WHERE {
    ?trace a <https://neuroshapes.org/SingleCellSimulationTrace> ;
           <https://neuroshapes.org/annotation> ?annotation .
    ?annotation a <https://bbp.epfl.ch/nexus/v1/resources/public/sscx/_/MTypeAnnotation> ;
                <https://neuroshapes.org/hasBody>/<http://www.w3.org/2000/01/rdf-schema#label> ?mtype.
}
""", limit=None)

In [8]:
premise = [
   {
        "type": "SparqlPremise",
        "hasParameter": [
            {
                "name": "BrainRegionQueryParameter",
                "description": "URI of the query brain region",
                "type": "uri"
            }
        ],
        "hasBody": """
            SELECT ?x
            WHERE {
                <$BrainRegionQueryParameter> <http://schema.org/isPartOf>* <http://api.brain-map.org/api/v2/data/Structure/315> .
            }
        """,
        "queryConfiguration": {
            "org": "neurosciencegraph",
            "project": "datamodels"
        }
    },
    {
        "type": "SparqlPremise",
        "hasParameter": [
            {
                "name": "MTypeQueryParameter",
                "description": "URI of the query MType",
                "type": "uri"
            }
        ],
        "hasBody": """
            SELECT ?x
            WHERE {
                <$MTypeQueryParameter> rdfs:subClassOf* <https://neuroshapes.org/PyramidalNeuron> .
            }
        """,
        "queryConfiguration": {
            "org": "neurosciencegraph",
            "project": "datamodels"
        }
    }
]

In [9]:
query = {
    "type": "QueryPipe",
    "head": {
        "type": "SparqlQuery",
        "hasBody": """
            SELECT ?brainRegion
            WHERE {
                ?brainRegion <http://schema.org/isPartOf>* <http://api.brain-map.org/api/v2/data/Structure/315> .
            }
        """,
        "queryConfiguration": {
            "org": "neurosciencegraph",
            "project": "datamodels"
          },
          "resultParameterMapping": {
            "parameterName": "acceptableBrainRegions",
            "path": "brainRegion"
          }
    },
    "rest": {
        "type": "QueryPipe",
        "head": {
            "type": "SparqlQuery",
            "hasBody": """
                SELECT ?mtype
                WHERE {
                    ?mtype rdfs:subClassOf* <https://neuroshapes.org/PyramidalNeuron> .
                }
            """,
            "queryConfiguration": {
                "org": "neurosciencegraph",
                "project": "datamodels"
            },
            "resultParameterMapping": {
                "parameterName": "acceptableMTypes",
                "path": "mtype"
            }
        },
        "rest": {
            "type": "SparqlQuery",
            "hasParameter": [
                {
                    "name": "acceptableBrainRegions",
                    "type": "uri_list",
                    "description": "List of acceptable brain regions"
                },
                {
                    "name": "acceptableMTypes",
                    "type": "uri_list",
                    "description": "List of acceptable mtypes"
                }
            ],
            "hasBody": """
                SELECT ?trace ?mtype
                WHERE {
                    ?trace a <https://neuroshapes.org/SingleCellSimulationTrace> ;
                           <https://neuroshapes.org/annotation> ?annotation ;
                           <https://neuroshapes.org/brainLocation>/<https://neuroshapes.org/brainRegion> ?brainRegion ;
                           <https://bluebrain.github.io/nexus/vocabulary/deprecated> false . 
                    ?annotation a <https://bbp.epfl.ch/nexus/v1/resources/public/sscx/_/MTypeAnnotation> ;
                                <https://neuroshapes.org/hasBody> ?mtype .
                    FILTER (?mtype IN ($acceptableMTypes)). 
                }
            """,
            "queryConfiguration": {
                "org": "public",
                "project": "sscx"
            }
        }
    }
}

In [10]:
rule = {
    "type": "DataGeneralizationRule",
    "name": "Simulation Trace generalization for cortical PCs",
    "description": "Collect smulation traces belonging to PCs from other cortical regions",
    "premise": premise,
    "searchQuery": query
}

In [11]:
input_filters = {
    "BrainRegionQueryParameter": "http://api.brain-map.org/api/v2/data/Structure/31",
    "MTypeQueryParameter": "http://uri.interlex.org/base/ilx_0381377",
    "SpeciesQueryParameter": "http://purl.obolibrary.org/obo/NCBITaxon_10088"
}

In [13]:
forge = allocate_forge_session("bbp", "inference-rules")

In [15]:
forge.register(forge.from_json(rule))

<action> _register_one
<succeeded> True


In [29]:
from kgforge import version

In [30]:
version.__version__

'0.7.1.dev10+gc0003f4'

In [31]:
forge.sparql()

<action> wrapper
<error> TypeError: sparql() missing 1 required positional argument: 'query'



In [32]:
apply_rule(allocate_forge_session, rule, input_filters)

<action> wrapper
<error> TypeError: sparql() got an unexpected keyword argument 'rewrite'

<action> as_json
<error> TypeError: 'NoneType' object is not iterable



TypeError: object of type 'NoneType' has no len()

In [56]:
len(mtypes)

18

In [ ]:
SELECT ?id
WHERE {
    ?id a <https://neuroshapes.org/SingleCellSimulationTrace> ;
        <https://neuroshapes.org/annotation> ?annotation ;
        <https://neuroshapes.org/brainLocation>/<https://neuroshapes.org/brainRegion> ?brainRegion.
        <https://bluebrain.github.io/nexus/vocabulary/deprecated> false .
    ?annotation a <https://bbp.epfl.ch/nexus/v1/resources/public/sscx/_/MTypeAnnotation> ;
                <https://neuroshapes.org/hasBody> ?mtype .
    ?mtype rdfs:subClassOf* <https://neuroshapes.org/PyramidalNeuron> .
    ?brainRegion <http://schema.org/isPartOf>* <http://api.brain-map.org/api/v2/data/Structure/315> .
}

In [ ]:
SELECT ?trace ?mtype
WHERE {
    ?trace a <https://neuroshapes.org/SingleCellSimulationTrace> ;
           <https://neuroshapes.org/annotation> ?annotation .
    ?annotation a <https://bbp.epfl.ch/nexus/v1/resources/public/sscx/_/MTypeAnnotation> ;
                <https://neuroshapes.org/hasBody>/<http://www.w3.org/2000/01/rdf-schema#label> ?mtype.
}

In [53]:
for label in pc_labels:
    print(label)
    print("""
    SELECT ?id
    WHERE {
        ?id a <http://www.w3.org/2002/07/owl#Class> ;
              <http://www.w3.org/2000/01/rdf-schema#label> "%s".
    }
    """ % label)
    r = nsg_forge.sparql(
    """
    SELECT ?id
    WHERE {
        ?id a <http://www.w3.org/2002/07/owl#Class> ;
              <http://www.w3.org/2000/01/rdf-schema#label> "%s".
    }
    """ % label)
    resource = nsg_forge.retrieve(r[0].id)

    resource.id = r[0].id
    resource.subClassOf = nsg_forge.from_json({
        "id": "https://neuroshapes.org/PyramidalNeuron",
        "type": "Class"
    })
    nsg_forge.update(resource)

L2_IPC

    SELECT ?id
    WHERE {
        ?id a <http://www.w3.org/2002/07/owl#Class> ;
              <http://www.w3.org/2000/01/rdf-schema#label> "L2_IPC".
    }
    
<action> _update_one
<succeeded> True
L2_TPC:A

    SELECT ?id
    WHERE {
        ?id a <http://www.w3.org/2002/07/owl#Class> ;
              <http://www.w3.org/2000/01/rdf-schema#label> "L2_TPC:A".
    }
    
<action> _update_one
<succeeded> True
L5_TPC:B

    SELECT ?id
    WHERE {
        ?id a <http://www.w3.org/2002/07/owl#Class> ;
              <http://www.w3.org/2000/01/rdf-schema#label> "L5_TPC:B".
    }
    
<action> _update_one
<succeeded> True
L5_UPC

    SELECT ?id
    WHERE {
        ?id a <http://www.w3.org/2002/07/owl#Class> ;
              <http://www.w3.org/2000/01/rdf-schema#label> "L5_UPC".
    }
    
<action> _update_one
<succeeded> True
L2_TPC:B

    SELECT ?id
    WHERE {
        ?id a <http://www.w3.org/2002/07/owl#Class> ;
              <http://www.w3.org/2000/01/rdf-schema#label> "L2_TPC:B".
  

In [23]:
for pair in resources:
    morph = sscx_forge.retrieve(pair.morph)
    trace = sscx_forge.retrieve(pair.trace)
    annotation = morph.annotation.hasBody.label
    if "PC" in annotation:    
        print(annotation)

L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:B
L5_TPC:A
L5_TPC:A
L5_TPC:B
L5_TPC:A
L5_TPC:A
L5_TPC:B
L5_TPC:A
L5_TPC:B
L5_TPC:A
L5_TPC:B
L5_TPC:A
L5_TPC:A
L5_TPC:A
L5_TPC:A
L5_TPC:A
L5_TPC:A
L5_TPC:A
L5_TPC:A
L5_TPC:A
L5_TPC:A
L5_TPC:A
L5_TPC:A
L5_TPC:A
L5_TPC:A
L5_TPC:A
L5_TPC:A
L5_TPC:A
L5_TPC:B
L

In [12]:
for r in resources:
    morphs = sscx_forge.search({
        "type": "NeuronMorphology"
    }, limit=None)

972

In [8]:
resources = sscx_forge.search({
    "type": "Trace",
    "derivation": {
        "entity": {
            "type": "PatchedCell"
        }
    }
}, limit=100)

In [9]:
print(resources[0])

{
    context:
    [
        https://bluebrain.github.io/nexus/contexts/metadata.json
        {
            type:
            {
                @id: rdf:type
            }
            label:
            {
                @id: rdfs:label
            }
            @vocab: https://bbp.epfl.ch/nexus/v1/resources/public/sscx/_/
            AcquisitionAnnotation:
            {
                @id: nsg:AcquisitionAnnotation
            }
            Activity:
            {
                @id: prov:Activity
            }
            AffineLinearTransform:
            {
                @id: nsg:AffineLinearTransform
            }
            Agent:
            {
                @id: prov:Agent
            }
            Analysis:
            {
                @id: nsg:Analysis
            }
            AnalysisConfiguration:
            {
                @id: nsg:AnalysisConfiguration
                @type: @id
            }
            AnalysisReport:
            {
                @id: nsg:Ana